In [ ]:
library(Seurat)
library(data.table)
library(ggplot2)
library(readxl)
library(dplyr)
library(ggplot2) 
library(viridis)
library(stringr)
library(lme4)


In [ ]:
setwd(".../Test_Code")

In [ ]:
datasets <- readRDS(".../RData/SEZ_seurat_object.rds")
#readRDS("SEZ_object.rds")# readRDS("SEZ_object.rds")
DefaultAssay(datasets) <- "RNA"
#Idents(datasets) <- datasets$cluster
cells<-levels(datasets)


In [ ]:
cells<-levels(datasets)


In [ ]:
group_info<-data.frame("case"=c("A","B","C","D","E","F","G","H"), "group"=c("Y","Y","Y","Y","M","M","M","M"))

# make a dataframe of the relative counts
rel_counts <- table(datasets$cluster, datasets$case) 
dg <- as.data.frame(rel_counts)
colnames(dg) <- c("cluster", "case", "counts")
dg$group <- group_info$group[match(dg$case, group_info$case)]

dg$id <- paste(dg$cluster, dg$group, sep = "-")

require(dplyr)
dg <- dg %>% group_by(case) %>% mutate(Percentage=round(counts/sum(counts)*100,4))
dg <- dg %>% group_by(case) %>% mutate(tot_counts=sum(counts))

dg <- dg %>% group_by(case) %>% mutate(Proportions=round(counts/sum(counts),4))

dg$not_counts<-dg$tot_counts-dg$counts
dg$group<-factor(dg$group)
dg$case<-factor(dg$case)

dg$case<-factor(dg$case)
length(levels(dg$case))
dg <- dg %>% mutate(group = relevel(group, "Y"))
dg$cluster<-factor(dg$cluster, levels = cells) 
dg<-with(dg, dg[order(cluster),])



dg_list <- split( dg, f = dg$cluster )

write.csv(dg,"counts_and_proportions.csv")

In [ ]:
stats<-dg %>% group_by(cluster) %>%  mutate(percentage=Proportions*100) %>% 
              summarise(mean_percentage = mean(percentage),  sd=sd(percentage))
sum(stats$mean_percentage)
write.csv(stats, "stats_all_clusters_case.csv")

In [ ]:
clusters<-cells
models_middleage_coef<-c()
models_p_value<-c()
models_cells<-list()
for (cluster in clusters){
    print(cluster)
    model <- glm(cbind(counts,not_counts) ~ group , family = quasibinomial,
          data = dg_list[[cluster]])#+ sorter
    models_cells[[cluster]]<-model
    middleage_coef=coef(model)[2]
    middleage_p_value=coef(summary(model))["groupM",4]
    models_middleage_coef=c(models_middleage_coef,middleage_coef)
    models_p_value=c(models_p_value,middleage_p_value)

             

    }#+sorter
#output
models_out<-data.frame(clusters)
models_out$middleage_coef<-models_middleage_coef
models_out$p_value<-models_p_value

models_out$bon_adj_pvalue<-p.adjust(models_p_value, method ="fdr", n = length(models_p_value))
rownames(models_out)<-clusters
models_out
write.csv(models_out, "glm_proportions_all_cluster_quasibinomial_fdr_case.csv")

In [ ]:
q <- ggplot(dg, aes(x=group, y=Proportions)) + 
     geom_boxplot(outlier.shape = NA,size=0.8)#geom_violin(trim=FALSE)#colour = "#999999",
q<-q+
    ylab("") + xlab("") +
    
    geom_jitter(aes(fill=case),size = 4.3,position = position_jitter(seed = 2,w = 0.4, h = 0),shape=21)+
    
    
    theme(strip.text = element_text(size=20),legend.key = element_rect(fill = "white"),
          panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
    panel.background = element_blank(), axis.line = element_line(colour = "black"),axis.text=element_text(size=17,color="black"),
          text=element_text( face = "bold" ,family="Arial", size=20),strip.background = element_blank(), axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),axis.text.y = element_text(size=18))+ # scale_y_continuous(limits = c(0, NA))

    facet_wrap(~cluster,scales = "free",ncol=2)


tiff("boxplot_proportions_all_cluster_donors.tiff", res = 400, units = "in", width = 7, height = 17.5)
q
dev.off()